In [1]:
import os
os.chdir("/nas/longleaf/home/cctsai/proj_dip/")
# /work/users/c/c/cctsai/data/BCP_sample/T2/

In [2]:
!jupyter nbconvert --to script self_supervised_beyond.ipynb

exp_namespace = 'T2/beyond/self_supervised'
submission_py = "self_supervised_beyond.py"
%run "self_supervised_beyond.ipynb"


# # # # # # 
"""

vvvvvvvvvvvvv
v) DIP+TV
v) dDIP, dDIP_noKspace (1.25)
v) sDIP, sDIP_noKspace (1.25, 1.6, 2)
v) without_kbound 但有kspace
v) kspace_mse_shape_without_boundary 
v) kspace_mse_shape
"""

"/nas/longleaf/home/cctsai"
"/work/users/c/c/cctsai"

[NbConvertApp] Converting notebook self_supervised_beyond.ipynb to script
[NbConvertApp] Writing 5941 bytes to self_supervised_beyond.py


usage: ipykernel_launcher.py [-h] [--file FILE] [--factor FACTOR]
                             [--model_folder MODEL_FOLDER] [--input_img]
                             [--double_arm DOUBLE_ARM]
ipykernel_launcher.py: error: unrecognized arguments: -f /nas/longleaf/home/cctsai/.local/share/jupyter/runtime/kernel-e3e908fa-2b65-420f-944e-34b8177b8e76.json


SystemExit: 2

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


SystemExit: 2

In [8]:
# You can change the names here
#exp_namespace = 'ablation/kspace_boundary/noMSE/{}'

### fixed ###
template = """#!/bin/bash

#SBATCH --job-name=pytorch
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=2
#SBATCH --mem=50G
#SBATCH --time=40:00:00
#SBATCH --partition=a100-gpu
#SBATCH --output=slurm_log/{}/run-%j.log --open-mode=append
#SBATCH --gres=gpu:1
#SBATCH --qos=gpu_access

# Path to the package directory
PACKAGE_PATH=/nas/longleaf/home/cctsai/.local/lib/python3.9/site-packages

# Add the package path to Python's sys.path
export PYTHONPATH="$PYTHONPATH:$PACKAGE_PATH"

module load cuda/11.8
module load python/3.9.6

cd /nas/longleaf/home/cctsai/proj_dip/

python {} {}
"""

In [9]:
# ages = ['17', '51', '54', '92', '109', '146', '179', '205', '233', '272', '317', '342', '357']
# ages = ['17', '179', '272', '317', '357']
ages = ['51', '179', '357']
# ages = ['233', '342', '205', '146', '317', '92', '17', '54', '109', '272']

import glob, os
files = []

for age in ages:
    files += glob.glob(f"/work/users/c/c/cctsai/data/BCP_sample/T2/*_{age}_*.npy")
print(files)

def create_slurm_folders(task_name):
    for dst_dir in ['slurm', 'slurm_log']:
        new_folder = f"/nas/longleaf/home/cctsai/proj_dip/{dst_dir}/{task_name}"
        if not os.path.exists(new_folder):
            os.makedirs(new_folder)
            print('Create:',new_folder)

['/work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP198549_51_T2w_SPC_NORM_5.npy', '/work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP198202_179_T2w_SPC_NORM_5.npy', '/work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP726893_357_T2w_SPC_NORM_5.npy']


In [10]:
# for factor in [2]:
#     for kbound_weight in [0.001, 0.0005, 0.0001]:
#         for kbound_outer in ['True','False']:
#             exp_name = exp_namespace.format(f'kbound_weight_{kbound_weight}/kbound_outer_{kbound_outer}')

factor = 2

# for kbound in ['kbound', 'nokbound']:
exp_name = f"{exp_namespace}"

###
task_name = exp_name
create_slurm_folders(task_name)
for npy_file in files:
    arg1 = f"--file {npy_file} --factor {factor} --model_folder {exp_name} "
    arg2 = f"--double_arm False"
    arg_strings = arg1 + arg2
    
    print(arg_strings)

    ### write to file
    script_content = template.format(task_name, submission_py, arg_strings)
    age = npy_file.split('/')[-1].split("_")[1]
    filename = f'/nas/longleaf/home/cctsai/proj_dip/slurm/{task_name}/age_{age}_.slurm'
    with open(filename, 'w') as script_file:
        script_file.write(script_content)

--file /work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP198549_51_T2w_SPC_NORM_5.npy --factor 2 --model_folder T2/beyond/self_supervised --double_arm False
--file /work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP198202_179_T2w_SPC_NORM_5.npy --factor 2 --model_folder T2/beyond/self_supervised --double_arm False
--file /work/users/c/c/cctsai/data/BCP_sample/T2/MNBCP726893_357_T2w_SPC_NORM_5.npy --factor 2 --model_folder T2/beyond/self_supervised --double_arm False


# Submit slurm files

In [11]:
slurm_files = []

# for tv_weight in [0.0005, 0.0001, 0.00005, 0.00001]:
#     exp_name = f"{exp_namespace}/tv_weight_{tv_weight}"


exp_name = f"{exp_namespace}"

# do not replace:
task_name = exp_name
for age in ages:
    slurm_files +=glob.glob(f"/nas/longleaf/home/cctsai/proj_dip/slurm/{task_name}/age_{age}_*.slurm")

print(len(slurm_files))
print(slurm_files)

3
['/nas/longleaf/home/cctsai/proj_dip/slurm/T2/beyond/self_supervised/age_51_.slurm', '/nas/longleaf/home/cctsai/proj_dip/slurm/T2/beyond/self_supervised/age_179_.slurm', '/nas/longleaf/home/cctsai/proj_dip/slurm/T2/beyond/self_supervised/age_357_.slurm']


In [12]:
import subprocess

stdouts = []

for slurm_file in slurm_files:
    command = f'sbatch {slurm_file}'
    completed_process = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    # Print the captured output
    print("Standard Output:", completed_process.stdout)
    stdouts.append(completed_process.stdout)
    
print("Total", len(stdouts), "jobs")

Standard Output: Submitted batch job 34762232

Standard Output: Submitted batch job 34762233

Standard Output: Submitted batch job 34762234

Total 3 jobs
